<a href="https://colab.research.google.com/github/tuomaseerola/msdemo/blob/main/demo_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
library(tidyverse)
library(readr)
library(knitr)

## Read Qualtrics data

In [61]:
myfile <- "https://raw.github.com/tuomaseerola/msdemo/main/filtered_data_wide.csv"
data <- read_csv(myfile)
d<-data.frame(data)
d$Progress<-as.numeric(d$Progress)
d2<-dplyr::filter(d,Progress>80) # Filter out the unfinished surveys
nrow(d2)
d2$Age <- as.numeric(d2$Age)
d2<-dplyr::filter(d,Age>17) # Filter out anybody under 18
nrow(d2)

Rows: 70 Columns: 49
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (37): Status, ResponseId, DistributionChannel, UserLanguage, Consent, G...
dbl   (5): Progress, Duration..in.seconds., LocationLatitude, LocationLongit...
lgl   (6): Finished, RecipientLastName, RecipientFirstName, RecipientEmail, ...
dttm  (1): RecordedDate

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 70

[1] 69

### Convert data to long format for easier analysis

In [62]:
# pivot
df <- pivot_longer(d2, cols = Q1:Q25, names_to = "Item", values_to = "Answer")

# Add stimulus details
dur <- c(150,150,150,150,150,250,250,250,250,250,500,500,500,500,500,1000,1000,1000,1000,1000,"full length","full length","full length","full length","full length")
emotion<-c("Fear","Anger","Tenderness","Happiness","Sadness","Sadness","Happiness","Tenderness","Fear","Anger","Anger","Tenderness","Fear","Sadness","Happiness","Fear","Happiness","Sadness","Tenderness","Anger","Sadness","Happiness","Tenderness","Anger","Fear")
df$Duration<-rep(dur,nrow(d2))
df$Correct<-rep(emotion,nrow(d2))
df$Correctness<-ifelse(df$Answer==df$Correct,1,0) # calculate correctness

#write_csv(df,"processed_data.csv") # export long data

# Explicitly code empty values as missing data (NA)
df$Answer[df$Answer==""]<-NA
df$Answer<-factor(df$Answer)
df$Correct<-factor(df$Correct)
df$Duration<-factor(df$Duration,levels = c(150,250,500,1000,"full length"))

## Analysis

### Confusion matrix


In [63]:
t <- table(df$Answer,df$Correct)
print(knitr::kable(t))

t<-table(df$Duration,df$Correctness)
knitr::kable(t[,2]/350,digits = 2)

t<-table(df$Correct,df$Correctness)
knitr::kable(t[,2]/350,digits = 2)



|           | Anger| Fear| Happiness| Sadness| Tenderness|
|:----------|-----:|----:|---------:|-------:|----------:|
|Anger      |   202|   27|        54|      14|          1|
|Fear       |   105|  263|        32|      40|          9|
|Happiness  |    16|   13|       203|      34|         44|
|Sadness    |     8|   20|        24|     198|        114|
|Tenderness |     1|    6|        23|      50|        165|




|            |    x|
|:-----------|----:|
|150         | 0.37|
|250         | 0.47|
|500         | 0.67|
|1000        | 0.70|
|full length | 0.74|



|           |    x|
|:----------|----:|
|Anger      | 0.58|
|Fear       | 0.75|
|Happiness  | 0.58|
|Sadness    | 0.57|
|Tenderness | 0.47|

Here are the results of the chi-square tests for the overall accuracy and for each emotion and duration separately. In the first calculation on Friday, I didn't adjust adjust the baseline probabilities of these chi-square tests. The probability of guessing correctly is 0.2 (1 was correct emotion out of 5 candidates), so I have altered the analyses to compare the fails (0) and successes (1) to the baseline of 0.8 (for fails) and 0.2 (for successes). You can see this in the code below

```chisq.test(t, p = c(0.8,0.2))```
where the `chisq.test` is the chi-square test function in R, `t` is the table of observed frequencies, and `p` is the vector of expected probabilities.

In [64]:
# Overall accuracy
t <- table(df$Correctness)
print(t)
# the probability of guessing correctly is 0.2
print(chisq.test(t,p = c(0.8,0.2)))

# Across all emotions
t <- table(df$Correct,df$Correctness)
chisq.test(t)

# for each emotion separately
chisq.test(t[1,], p = c(0.8,0.2)) # Anger
chisq.test(t[2,], p = c(0.8,0.2)) # Fear
chisq.test(t[3,], p = c(0.8,0.2)) # Happiness
chisq.test(t[4,], p = c(0.8,0.2)) # Sadness
chisq.test(t[5,], p = c(0.8,0.2)) # Tenderness


   0    1 
 635 1031 

	Chi-squared test for given probabilities

data:  t
X-squared = 1826.7, df = 1, p-value < 2.2e-16




	Pearson's Chi-squared test

data:  t
X-squared = 68.651, df = 4, p-value = 4.372e-14



	Chi-squared test for given probabilities

data:  t[1, ]
X-squared = 346.15, df = 1, p-value < 2.2e-16



	Chi-squared test for given probabilities

data:  t[2, ]
X-squared = 738.75, df = 1, p-value < 2.2e-16



	Chi-squared test for given probabilities

data:  t[3, ]
X-squared = 343.04, df = 1, p-value < 2.2e-16



	Chi-squared test for given probabilities

data:  t[4, ]
X-squared = 318.24, df = 1, p-value < 2.2e-16



	Chi-squared test for given probabilities

data:  t[5, ]
X-squared = 181.73, df = 1, p-value < 2.2e-16



### Across durations

In [65]:
# Across all durations
t <- table(df$Duration,df$Correctness)
chisq.test(t)

# Across each duration separately
# all significant below p<0.001 level!
chisq.test(t[1,], p = c(0.8,0.2)) # 150ms
chisq.test(t[2,], p = c(0.8,0.2)) # 250ms
chisq.test(t[3,], p = c(0.8,0.2)) # 500ms
chisq.test(t[4,], p = c(0.8,0.2)) # 1000ms
chisq.test(t[5,], p = c(0.8,0.2)) # full length


	Pearson's Chi-squared test

data:  t
X-squared = 169.17, df = 4, p-value < 2.2e-16



	Chi-squared test for given probabilities

data:  t[1, ]
X-squared = 75.442, df = 1, p-value < 2.2e-16



	Chi-squared test for given probabilities

data:  t[2, ]
X-squared = 173.06, df = 1, p-value < 2.2e-16



	Chi-squared test for given probabilities

data:  t[3, ]
X-squared = 505.86, df = 1, p-value < 2.2e-16



	Chi-squared test for given probabilities

data:  t[4, ]
X-squared = 594.22, df = 1, p-value < 2.2e-16



	Chi-squared test for given probabilities

data:  t[5, ]
X-squared = 727.5, df = 1, p-value < 2.2e-16


### Across musicianship levels

In [66]:
t <- table(df$Musicianship,df$Correctness)
knitr::kable(t[,2]/rowSums(t),digits = 2)

# recode musicians into non-musicians and musicians
df$Musicianship <- factor(df$Musicianship,levels=c("Nonmusician","Music-loving nonmusician","Amateur musician","Serious amateur musician","Semi-professional musician"),labels = c("Non-musician","Non-musician","Musician","Musician","Musician"))
t <- table(df$Musicianship,df$Correctness)
knitr::kable(t[,2]/rowSums(t),digits = 2)

chisq.test(t[1,], p = c(0.8,0.2)) # non-musicians
chisq.test(t[2,], p = c(0.8,0.2)) # musicians



|                           |    x|
|:--------------------------|----:|
|Amateur musician           | 0.66|
|Music-loving nonmusician   | 0.57|
|Nonmusician                | 0.63|
|Semi-professional musician | 0.56|
|Serious amateur musician   | 0.66|



|             |    x|
|:------------|----:|
|Non-musician | 0.58|
|Musician     | 0.66|


	Chi-squared test for given probabilities

data:  t[1, ]
X-squared = 686.56, df = 1, p-value < 2.2e-16



	Chi-squared test for given probabilities

data:  t[2, ]
X-squared = 1155.9, df = 1, p-value < 2.2e-16
